# Grab Dimensions Data

https://app.dimensions.ai/discover/publication

This notebook handles the downloads of Dimensions data.


## Search Params

search_params logs what you want to search from Dimensions.

In [1]:
# This is very important for data fetching, generally you get url in the form as 'https://app.dimensions.ai/discover/publication?and_facet_for=2209&and_facet_for=2202&or_facet_source_title=jour.1082997', and search_params is just the substring after '?'.
# Here is where we need to change for each search.
search_params = 'or_facet_year=2014&or_facet_year=2015&or_facet_year=2016&or_facet_source_title=jour.1018957'

In [2]:
origin_folder = 'Journals'

## Request Functions

This part contains functions we need to fetch the web data and should also handle the exceptions while fetching here.

In [8]:
import requests
import json

from requests import ConnectionError, ReadTimeout

def grab_from_url_content(url):
    headers = {'Accept': '* / *',
               'Accept-Language': 'zh-TW, zh; q=0.9, en-US; q=0.8, en; q=0.7, zh-CN; q=0.6',
               'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3610.2 Safari/537.36'
               }
    rescontent = ''
    try:
        res = requests.get(url, headers=headers, timeout=10)
        rescontent = res.text
    except ConnectionError as ce:
        print('ConnectionError: ' + str(ce))
        return grab_from_url_content(url)
    except ReadTimeout as rte:
        print('ReadTimeout: ' + str(rte))
        return grab_from_url_content(url)

    return rescontent



## Parser

We will get the html content from the url which is not listed as we want it be, so we need parser to parse them into listed data, in json form.

In [3]:
from html.parser import HTMLParser

class DimensionsHTMLParser(HTMLParser):
    articles = []
    in_article = False
    is_next_trigger = False
    next_trigger_url = ''

    def handle_starttag(self, tag, attrs):
        if tag == 'article':
            self.in_article = True
        
        if self.in_article:
            for attr in attrs:
                if attr[0] == 'data-doc':
                    self.articles.append(json.loads(attr[1]))
                    break

        if tag == 'a':
            for attr in attrs:
                if (attr[0] == 'class') and (attr[1] == 'nextPage-trigger'):
                    self.is_next_trigger = True
                if self.is_next_trigger and (attr[0] == 'href'):
                    self.next_trigger_url = f'https://app.dimensions.ai{attr[1]}&{search_params}'
                    break
        return
                    
    def handle_endtag(self, tag):
        if tag == 'article':
            self.in_article = False
        self.is_next_trigger = False
        return

    def handle_data(self, data):
        pass

    def handle_comment(self, data):
        pass

    def handle_entityref(self, name):
        pass

    def handle_charref(self, name):
        pass

    def handle_decl(self, data):
        pass

parser = DimensionsHTMLParser()

## Grab Data

The data fetching starts from here.

In [4]:
grab_url = f'https://app.dimensions.ai/discover/publication?{search_params}'

In [5]:
%%time
parser.feed(grab_from_url_content(grab_url))
next_trigger_anchor = ''
i = 0
while parser.next_trigger_url != next_trigger_anchor:
    if i % 100 == 0:
        print(str(i))
    i+=1
    next_trigger_anchor = parser.next_trigger_url
    parser.feed(grab_from_url_content(parser.next_trigger_url))

0
100
ReadTimeout: HTTPSConnectionPool(host='app.dimensions.ai', port=443): Read timed out. (read timeout=10)
200
300
ReadTimeout: HTTPSConnectionPool(host='app.dimensions.ai', port=443): Read timed out. (read timeout=10)
ReadTimeout: HTTPSConnectionPool(host='app.dimensions.ai', port=443): Read timed out. (read timeout=10)
400
500
ReadTimeout: HTTPSConnectionPool(host='app.dimensions.ai', port=443): Read timed out. (read timeout=10)
600
ReadTimeout: HTTPSConnectionPool(host='app.dimensions.ai', port=443): Read timed out. (read timeout=10)
CPU times: user 29 s, sys: 1.73 s, total: 30.7 s
Wall time: 19min 22s


In [6]:
%%time
print(len(parser.articles))

12226
CPU times: user 400 µs, sys: 656 µs, total: 1.06 ms
Wall time: 876 µs


In [7]:
with open(f"{origin_folder}/article_nature.json", "w") as dump_f:
        dump_f.write(json.dumps(parser.articles))

In [15]:
# import pandas as pd

# with open(f'{origin_folder}/article_nature.json') as f:
#     articles = json.load(f)

# df_articles = pd.DataFrame(columns=['SO', 'SO_id', 'DI', 'dimensions_id'])

for article in articles:
    if 'doi' not in article:
        print(article)
        continue
    if article['doi'] in df_articles['DI']:
        continue
    df_article = pd.DataFrame(columns=['SO', 'SO_id', 'DI', 'dimensions_id'])
    df_article['SO'] = [article['source_title']]
    df_article['SO_id'] = [article['source_title_id']]
    df_article['DI'] = [article['doi']]
    df_article['dimensions_id'] = [article['id']]
    df_articles = df_articles.append(df_article, ignore_index=True)

df_articles.to_csv(f'{origin_folder}/article_nature.csv', index=False)

{'title': 'Boosting business.', 'affiliations_json': '[{"first_name": "Kendall", "last_name": "Powell", "corresponding": "", "orcid": "", "current_organization_id": "", "researcher_id": "", "affiliations": [], "raw_affiliation": []}]', 'author_list': 'Kendall Powell', 'source_title_id': 'jour.1018957', 'journal_title': 'Nature', 'sn_journal_open_access_option': 'Subscription Only', 'language': 'en', 'open_access': False, 'pub_class': 'Article', 'abstract': '', 'acknowledgements': '', 'aff_country_count': 0, 'aff_org_count': 0, 'created_in_dimensions': '2017-08-31T12:50:56Z', 'html_escaped_abstract': '', 'html_escaped_title': 'Boosting business.', 'id': 'pub.1079016901', 'issue': '7529', 'pages': '133-5', 'pmid': '25513444', 'pub_date': '2014-12-04', 'pub_year': 2014, 'volume': '516', 'sn_downloads_customer_1': 0, 'times_cited': 0, 'sn_denials_customer_1': 0, 'sn_denials': 0, 'sn_downloads': 0, 'altmetric_id': 0, 'score': 1.0, 'affiliations_details': [{'first_name': 'Kendall', 'last_nam